[colab](https://colab.research.google.com/github/rufous86/studies/blob/main/DLS_NLP/02_hw_embedding_based_word_ranking/solution_simple_embeddings.ipynb)

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [60]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-05-16 07:39:16--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  2.57MB/s    in 10m 12s 

2023-05-16 07:49:33 (2.27 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [61]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [62]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [63]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [64]:
# method most_simmilar
'''your code'''
import numpy as np
import pandas as pd

dog_embedding = wv_embeddings['dog']
cosine_with_dog = {}
for word in wv_embeddings.index_to_key:
    cosine = np.dot(dog_embedding, wv_embeddings[word]).T/(np.linalg.norm(dog_embedding)*np.linalg.norm(wv_embeddings[word]))
    cosine_with_dog[word] = cosine

cosine_with_dog_df = pd.DataFrame(cosine_with_dog, index=['cosine']).T.sort_values(by='cosine', ascending=False).reset_index()
cosine_with_dog_df.iloc[1:6, :]

,index,cosine
1,animal,0.856418
2,dogs,0.788087
3,mammal,0.762380
4,cats,0.762125
5,animals,0.760794


In [65]:
cosine_with_dog_df.loc[cosine_with_dog_df['index'] == 'cat', :]

,index,cosine
26,cat,0.685234


Слово слов `cat` не входит в топ-5 близких слов к слову `dog`. Оно находится на 26-м месте

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [66]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [67]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    '''your code'''
    tokens = tokenizer().tokenize(question)
    words_in_question = []
    for token in tokens:
        if token in embeddings:
            words_in_question.append(embeddings[token])
    if not words_in_question:
        return np.zeros((dim,))
    return np.mean(words_in_question, axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [103]:
'''your code'''
from nltk.tokenize import WordPunctTokenizer

question_to_vec('I love neural networks', 
                wv_embeddings,
                WordPunctTokenizer).round(2)[2]

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Ответ: Максимум (`Hits@47 - DCG@1`) = максимум `Hits@47` - минимум `DCG@1` = 1 - 0 = 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [69]:
import math

round(1 / math.log2(9+1), 1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [70]:
from math import log2

def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    '''your code'''
    hits_value = [int(dup_rank<=k) for dup_rank in dup_ranks]
    return sum(hits_value) / len(hits_value)

In [71]:
def dcg_score(dup_ranks, k): 
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    '''your code'''
    
    dcg_value = [(1 / log2(dup_rank+1)*int(dup_rank<=k)) for dup_rank in dup_ranks]
    return sum(dcg_value) / len(dcg_value)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [72]:
import pandas as pd
import numpy as np

In [73]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [74]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [75]:
from google.colab import drive
drive.mount('/content/drive')
! cp '/content/drive/MyDrive/Study/courses/DLS - NLP/2. [HW] Ранжирование слов на основе эмбедингов/stackoverflow_similar_questions.zip' .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Считайте данные.

In [77]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        '''your code'''
        data.append(line)
    return data

Нам понадобиться только файл validation.

In [78]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [79]:
len(validation_data)

3760

Размер нескольких первых строк

In [80]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54517
2 52818
3 54464
4 53724
5 52297


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [81]:
a = np.array([1, 2, 3]).reshape(1, -1)
b = np.array([1, 2, 3]).reshape(1, -1)
# cosine_similarity(a, b)[0][0]
# d = {}
# d[cosine_similarity(a, b)[0][0]] = (a, b)

In [82]:
d[cosine_similarity(a, b)[0][0]] = (a, b)
d

{0.8976906560180993: (array([[1, 2, 3]]), array([[ 1,  2, 12]])),
 0.8728715609439697: (array([[1, 2, 3]]), array([[1, 2, 1]])),
 1.0: (array([[1, 2, 3]]), array([[1, 2, 3]]))}

In [83]:
(np.array((sorted(d.items())))[:,1]).tolist()

<ipython-input-83-9450950a5c5e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  (np.array((sorted(d.items())))[:,1]).tolist()


[(array([[1, 2, 3]]), array([[1, 2, 1]])),
 (array([[1, 2, 3]]), array([[ 1,  2, 12]])),
 (array([[1, 2, 3]]), array([[1, 2, 3]]))]

In [84]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [127]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    '''your code'''
    q_vec = question_to_vec(question, embeddings, tokenizer, dim)
    cand_vecs = []
    for candidate in candidates:
        cand_vecs.append(question_to_vec(candidate, embeddings, tokenizer, dim))

    cand_dict = {}
    for i, cand_vec in enumerate(cand_vecs):
        cand_dict[cosine_similarity(q_vec.reshape(1, -1), cand_vec.reshape(1, -1))[0][0]] = (i, candidates[i])
    return [(np.array((sorted(cand_dict.items(), reverse=True)))[:,1]).tolist()]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [111]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [128]:
tokenizer = WordPunctTokenizer
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]]

[[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Ответ: `102`

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [118]:
from tqdm.notebook import tqdm

In [133]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    print(ex)
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

['o', 'w', ' ', 't', 'o', ' ', 'p', 'r', 'i', 'n', 't', ' ', 'a', ' ', 'b', 'i', 'n', 'a', 'r', 'y', ' ', 'h', 'e', 'a', 'p', ' ', 't', 'r', 'e', 'e', ' ', 'w', 'i', 't', 'h', 'o', 'u', 't', ' ', 'r', 'e', 'c', 'u', 'r', 's', 'i', 'o', 'n', '?', '\t', 'H', 'o', 'w', ' ', 'd', 'o', ' ', 'y', 'o', 'u', ' ', 'b', 'e', 's', 't', ' ', 'c', 'o', 'n', 'v', 'e', 'r', 't', ' ', 'a', ' ', 'r', 'e', 'c', 'u', 'r', 's', 'i', 'v', 'e', ' ', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n', ' ', 't', 'o', ' ', 'a', 'n', ' ', 'i', 't', 'e', 'r', 'a', 't', 'i', 'v', 'e', ' ', 'o', 'n', 'e', '?', '\t', 'H', 'o', 'w', ' ', 'c', 'a', 'n', ' ', 'i', ' ', 'u', 's', 'e', ' ', 'n', 'g', '-', 'm', 'o', 'd', 'e', 'l', ' ', 'w', 'i', 't', 'h', ' ', 'd', 'i', 'r', 'e', 'c', 't', 'i', 'v', 'e', ' ', 'i', 'n', ' ', 'a', 'n', 'g', 'u', 'l', 'a', 'r', ' ', 'j', 's', '\t', 'f', 'l', 'a', 's', 'h', ':', ' ', 'd', 'r', 'a', 'w', 'i', 'n', 'g', ' ', 'a', 'n', 'd', ' ', 'e', 'r', 'a', 's', 'i', 'n', 'g', '\t', 't', 'o', 'g', 'g', 

ValueError: ignored

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
'''your code'''

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count='''your code''',             # consider words that occured at least 5 times
                 window='''your code''').wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index('0') + 1)

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
